In [1185]:
import pandas as pd
import numpy as np

#features
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

#models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble  import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

#CV
from sklearn.model_selection import cross_validate

#GridSearch
from sklearn.model_selection import GridSearchCV


https://www.scitepress.org/Papers/2021/104468/104468.pdf

In [1186]:
original_df = pd.read_csv("fakeWhatsApp.BR_2018.csv")

A:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [1187]:
original_df = original_df[original_df['midia']==0]
original_df = original_df[original_df['viral']==1]
original_df.shape

(20872, 19)

In [1188]:
original_df = original_df.drop_duplicates(subset=['text'])
original_df.shape

(5284, 19)

In [1189]:
text_df = original_df[["text","misinformation"]]

In [1190]:
text_df["misinformation"].value_counts()

 0    2547
 1    2041
-1     696
Name: misinformation, dtype: int64

In [1191]:
text_df = text_df[text_df["misinformation"]>-1]

In [1192]:
X_train, X_test, y_train, y_test = train_test_split(text_df['text'], text_df['misinformation'], test_size=0.2, 
                                                                    stratify = text_df['misinformation'], random_state=42)

## Vetorizações

In [1193]:
#BOW                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
vectorizer_bow = CountVectorizer(binary=True, ngram_range = (1,1))
bow_transformer = vectorizer_bow.fit(X_train)
X_bow_train = bow_transformer.transform(X_train)

In [1194]:
#TFIDF
vectorizer_tfidf = TfidfVectorizer(ngram_range = (1,1))
tfidf_transformer = vectorizer_tfidf.fit(X_train)
X_tfidf_train = tfidf_transformer.transform(X_train)

## Modelos

In [1195]:
SEED = 42

In [1196]:
scores = ['accuracy','f1','precision','recall','roc_auc']

scores_df = pd.DataFrame(columns = ['modelo','vetorizacao','acuracia media','acuracia desvio',
               'f1 media','f1 desvio',
               'precisao media','precisao desvio',
               'revocacao media','revocacao desvio',
               'auc media','auc desvio'])

## Regressao Logistica

### BOW

In [1197]:
logreg = LogisticRegression()
results_cv = cross_validate(logreg, X_bow_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'LogisticRegression','vetorizacao':'BOW',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)

### TFIDF

In [1198]:
logreg = LogisticRegression()
results_cv = cross_validate(logreg, X_tfidf_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'LogisticRegression','vetorizacao':'TFIDF',
                              'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                               'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                               'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                               'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                               'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)

## Complement Naive Bayes

### BOW

In [1199]:
cnb = ComplementNB()
results_cv = cross_validate(cnb, X_bow_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'ComplementNB','vetorizacao':'BOW',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)

### TFIDF

In [1200]:
cnb = ComplementNB()
results_cv = cross_validate(cnb, X_tfidf_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'ComplementNB','vetorizacao':'TFIDF',
                              'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                               'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                               'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                               'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                               'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)

## Naive Bayes de Bernoulli

### BOW

In [1201]:
parameters = {
    'alpha': np.linspace(0.05, 1.25, 10000)
}

grid_search = GridSearchCV( BernoulliNB(), 
                            parameters,
                            n_jobs = -1,    
                            scoring = scores,
                            cv = 10,
                            refit='accuracy',
                            verbose=3
)

grid_search.fit(X_bow_train, y_train)

scores_df = scores_df.append({'modelo':'BernoulliNB','vetorizacao':'BOW',
                                  'acuracia media': grid_search.cv_results_['mean_test_accuracy'].mean(),'acuracia desvio': grid_search.cv_results_['std_test_accuracy'].std(),
                                   'f1 media': grid_search.cv_results_['mean_test_f1'].mean(), 'f1 desvio': grid_search.cv_results_['std_test_f1'].std(),
                                   'precisao media': grid_search.cv_results_['mean_test_precision'].mean(),'precisao desvio': grid_search.cv_results_['std_test_precision'].std(),
                                   'revocacao media': grid_search.cv_results_['mean_test_recall'].mean(),'revocacao desvio': grid_search.cv_results_['std_test_recall'].std(),
                                   'auc media': grid_search.cv_results_['mean_test_roc_auc'].mean(),'auc desvio': grid_search.cv_results_['std_test_roc_auc'].std()},ignore_index=True)

Fitting 10 folds for each of 10000 candidates, totalling 100000 fits


### TFIDF

In [1202]:
parameters = {
     'alpha': np.linspace(0.05, 1.25, 10000)
}

grid_search = GridSearchCV( BernoulliNB(), 
                            parameters,
                            n_jobs = -1,    
                            scoring = scores,
                            cv = 10,
                            refit='accuracy',
                            verbose=3
)

grid_search.fit(X_tfidf_train, y_train)

scores_df = scores_df.append({'modelo':'BernoulliNB','vetorizacao':'TFIDF',
                                  'acuracia media': grid_search.cv_results_['mean_test_accuracy'].mean(),'acuracia desvio': grid_search.cv_results_['std_test_accuracy'].std(),
                                   'f1 media': grid_search.cv_results_['mean_test_f1'].mean(), 'f1 desvio': grid_search.cv_results_['std_test_f1'].std(),
                                   'precisao media': grid_search.cv_results_['mean_test_precision'].mean(),'precisao desvio': grid_search.cv_results_['std_test_precision'].std(),
                                   'revocacao media': grid_search.cv_results_['mean_test_recall'].mean(),'revocacao desvio': grid_search.cv_results_['std_test_recall'].std(),
                                   'auc media': grid_search.cv_results_['mean_test_roc_auc'].mean(),'auc desvio': grid_search.cv_results_['std_test_roc_auc'].std()},ignore_index=True)

Fitting 10 folds for each of 10000 candidates, totalling 100000 fits


## Naive Bayes Multinominal

### BOW

In [1203]:
parameters = {
    'alpha': np.linspace(1, 1, 1)
}

grid_search = GridSearchCV( MultinomialNB(), 
                            parameters,
                            n_jobs = -1,    
                            scoring = scores,
                            cv = 10,
                            refit='accuracy',
                            verbose=3
)

grid_search.fit(X_bow_train, y_train)

scores_df = scores_df.append({'modelo':'MultinomialNB','vetorizacao':'BOW',
                                  'acuracia media': grid_search.cv_results_['mean_test_accuracy'].mean(),'acuracia desvio': grid_search.cv_results_['std_test_accuracy'].std(),
                                   'f1 media': grid_search.cv_results_['mean_test_f1'].mean(), 'f1 desvio': grid_search.cv_results_['std_test_f1'].std(),
                                   'precisao media': grid_search.cv_results_['mean_test_precision'].mean(),'precisao desvio': grid_search.cv_results_['std_test_precision'].std(),
                                   'revocacao media': grid_search.cv_results_['mean_test_recall'].mean(),'revocacao desvio': grid_search.cv_results_['std_test_recall'].std(),
                                   'auc media': grid_search.cv_results_['mean_test_roc_auc'].mean(),'auc desvio': grid_search.cv_results_['std_test_roc_auc'].std()},ignore_index=True)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


### TFIDF

In [1204]:
parameters = {
    'alpha': np.linspace(1, 1, 1)
}

grid_search = GridSearchCV( MultinomialNB(), 
                            parameters,
                            n_jobs = -1,    
                            scoring = scores,
                            cv = 10,
                            refit='accuracy',
                            verbose=3
)

grid_search.fit(X_tfidf_train, y_train)

scores_df = scores_df.append({'modelo':'MultinomialNB','vetorizacao':'TFIDF',
                                  'acuracia media': grid_search.cv_results_['mean_test_accuracy'].mean(),'acuracia desvio': grid_search.cv_results_['std_test_accuracy'].std(),
                                   'f1 media': grid_search.cv_results_['mean_test_f1'].mean(), 'f1 desvio': grid_search.cv_results_['std_test_f1'].std(),
                                   'precisao media': grid_search.cv_results_['mean_test_precision'].mean(),'precisao desvio': grid_search.cv_results_['std_test_precision'].std(),
                                   'revocacao media': grid_search.cv_results_['mean_test_recall'].mean(),'revocacao desvio': grid_search.cv_results_['std_test_recall'].std(),
                                   'auc media': grid_search.cv_results_['mean_test_roc_auc'].mean(),'auc desvio': grid_search.cv_results_['std_test_roc_auc'].std()},ignore_index=True)


Fitting 10 folds for each of 1 candidates, totalling 10 fits


## SVM Linear

### BOW

In [1205]:
svm = LinearSVC()
results_cv = cross_validate(svm, X_bow_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'LinearSVC','vetorizacao':'BOW',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)

A:\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
A:\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
A:\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
A:\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
A:\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


### TFIDF

In [1206]:
svm = LinearSVC()
results_cv = cross_validate(mnb, X_tfidf_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'LinearSVC','vetorizacao':'TFIDF',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)

## Linear SVM with SGD training

### BOW

In [1207]:
sgd = SGDClassifier()
results_cv = cross_validate(sgd, X_bow_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'SGDClassifier','vetorizacao':'BOW',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)

### TFIDF

In [1208]:
sgd = SGDClassifier()
results_cv = cross_validate(sgd, X_tfidf_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'SGDClassifier','vetorizacao':'TFIDF',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)

## RBF SVM

### BOW

In [1209]:
'''
svc = SVC()
results_cv = cross_validate(svc, X_bow_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'SVC','vetorizacao':'BOW',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)
                                   '''

"\nsvc = SVC()\nresults_cv = cross_validate(svc, X_bow_train, y_train, cv=10,scoring=scores)\nscores_df = scores_df.append({'modelo':'SVC','vetorizacao':'BOW',\n                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),\n                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),\n                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),\n                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),\n                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)\n                                   "

### TFIDF

In [1210]:
'''
svc = SVC()
results_cv = cross_validate(svc, X_tfidf_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'SVC','vetorizacao':'TFIDF',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)
                                   '''

"\nsvc = SVC()\nresults_cv = cross_validate(svc, X_tfidf_train, y_train, cv=10,scoring=scores)\nscores_df = scores_df.append({'modelo':'SVC','vetorizacao':'TFIDF',\n                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),\n                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),\n                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),\n                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),\n                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)\n                                   "

## KNN

### BOW

In [1211]:
knn = KNeighborsClassifier(n_jobs=-1)
results_cv = cross_validate(knn, X_bow_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'KNeighborsClassifier','vetorizacao':'BOW',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)

### TFIDF

In [1212]:
knn = KNeighborsClassifier(n_jobs=-1)
results_cv = cross_validate(knn, X_tfidf_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'KNeighborsClassifier','vetorizacao':'TFIDF',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)

## Random Forest

### BOW

In [1213]:
'''
rf = RandomForestClassifier()
results_cv = cross_validate(rf, X_bow_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'RandomForestClassifier','vetorizacao':'BOW',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)'''

"\nrf = RandomForestClassifier()\nresults_cv = cross_validate(rf, X_bow_train, y_train, cv=10,scoring=scores)\nscores_df = scores_df.append({'modelo':'RandomForestClassifier','vetorizacao':'BOW',\n                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),\n                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),\n                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),\n                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),\n                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)"

### TFIDF

In [1214]:
'''
rf = RandomForestClassifier()
results_cv = cross_validate(rf, X_tfidf_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'RandomForestClassifier','vetorizacao':'TFIDF',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)'''

"\nrf = RandomForestClassifier()\nresults_cv = cross_validate(rf, X_tfidf_train, y_train, cv=10,scoring=scores)\nscores_df = scores_df.append({'modelo':'RandomForestClassifier','vetorizacao':'TFIDF',\n                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),\n                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),\n                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),\n                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),\n                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)"

## Gradient Boosting

### BOW

In [1215]:
'''
gb = GradientBoostingClassifier()
results_cv = cross_validate(gb, X_bow_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'GradientBoostingClassifier','vetorizacao':'BOW',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)
                '''

"\ngb = GradientBoostingClassifier()\nresults_cv = cross_validate(gb, X_bow_train, y_train, cv=10,scoring=scores)\nscores_df = scores_df.append({'modelo':'GradientBoostingClassifier','vetorizacao':'BOW',\n                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),\n                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),\n                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),\n                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),\n                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)\n                "

### TFIDF

In [1216]:
'''
gb = GradientBoostingClassifier()
results_cv = cross_validate(gb, X_tfidf_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'GradientBoostingClassifier','vetorizacao':'TFIDF',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)
                                   '''

"\ngb = GradientBoostingClassifier()\nresults_cv = cross_validate(gb, X_tfidf_train, y_train, cv=10,scoring=scores)\nscores_df = scores_df.append({'modelo':'GradientBoostingClassifier','vetorizacao':'TFIDF',\n                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),\n                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),\n                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),\n                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),\n                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)\n                                   "

## Multilayer Perceptron

### BOW

In [1217]:
'''
mlp = MLPClassifier(max_iter = 5, early_stopping= True)
results_cv = cross_validate(mlp, X_bow_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'MLPClassifier','vetorizacao':'BOW',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)
                                '''

"\nmlp = MLPClassifier(max_iter = 5, early_stopping= True)\nresults_cv = cross_validate(mlp, X_bow_train, y_train, cv=10,scoring=scores)\nscores_df = scores_df.append({'modelo':'MLPClassifier','vetorizacao':'BOW',\n                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),\n                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),\n                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),\n                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),\n                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)\n                                "

### TFIDF

In [1218]:
'''
mlp = MLPClassifier(max_iter = 5, early_stopping= True)
results_cv = cross_validate(mlp, X_tfidf_train, y_train, cv=10,scoring=scores)
scores_df = scores_df.append({'modelo':'MLPClassifier','vetorizacao':'TFIDF',
                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),
                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),
                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),
                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),
                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)
                                   '''

"\nmlp = MLPClassifier(max_iter = 5, early_stopping= True)\nresults_cv = cross_validate(mlp, X_tfidf_train, y_train, cv=10,scoring=scores)\nscores_df = scores_df.append({'modelo':'MLPClassifier','vetorizacao':'TFIDF',\n                                  'acuracia media':results_cv['test_accuracy'].mean(),'acuracia desvio':results_cv['test_accuracy'].std(),\n                                   'f1 media':results_cv['test_f1'].mean(),'f1 desvio':results_cv['test_f1'].std(),\n                                   'precisao media':results_cv['test_precision'].mean(),'precisao desvio':results_cv['test_precision'].std(),\n                                   'revocacao media':results_cv['test_recall'].mean(),'revocacao desvio':results_cv['test_recall'].std(),\n                                   'auc media':results_cv['test_roc_auc'].mean(),'auc desvio':results_cv['test_roc_auc'].std()},ignore_index=True)\n                                   "

# Melhores Resultados (ate aqui)

In [1219]:
scores_df.sort_values(by='f1 media',ascending=False)

,modelo,vetorizacao,acuracia media,acuracia desvio,f1 media,f1 desvio,precisao media,precisao desvio,revocacao media,revocacao desvio,auc media,auc desvio
3,ComplementNB,TFIDF,0.739782,0.017287,0.744887,0.015510,0.661143,0.018137,0.853681,0.025994,0.849126,0.016546
7,MultinomialNB,TFIDF,0.755858,0.000000,0.732169,0.000000,0.715155,0.000000,0.750793,0.000000,0.849126,0.000000
9,LinearSVC,TFIDF,0.755858,0.019390,0.732169,0.023061,0.715155,0.020845,0.750793,0.034825,0.849126,0.016546
11,SGDClassifier,TFIDF,0.764850,0.024766,0.731970,0.026719,0.743709,0.033104,0.721420,0.031544,0.843386,0.012319
6,MultinomialNB,BOW,0.738420,0.000000,0.729930,0.000000,0.675654,0.000000,0.794254,0.000000,0.828275,0.000000
2,ComplementNB,BOW,0.734332,0.020832,0.729815,0.018807,0.667361,0.023620,0.805888,0.023679,0.828282,0.019037
0,LogisticRegression,BOW,0.766485,0.015702,0.728089,0.018910,0.755859,0.022778,0.703019,0.027779,0.847032,0.014468
10,SGDClassifier,BOW,0.754768,0.010049,0.721062,0.015648,0.729829,0.012158,0.713415,0.031749,0.828719,0.011343
1,LogisticRegression,TFIDF,0.770027,0.014377,0.717767,0.016311,0.791320,0.024152,0.657096,0.017880,0.849702,0.017979
8,LinearSVC,BOW,0.742234,0.015944,0.708173,0.016381,0.714711,0.026383,0.703004,0.027690,0.814157,0.013689
